In [1]:
from sklearn.neural_network import MLPClassifier as mlp
import sys
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML")
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML/FeH")
from fitsParser.fitsParser import fitsParser
import pandas as pd
import numpy as np
import re
from numpy import linspace
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from modelValidator import psMetric, cut, shuffleDwarfs, plotRes, plot, norm_inputs, monte_carlo, printTable, conf_matrix
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from function import save_data
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Nadam
from sklearn.cluster import KMeans
binsize = 0.5

def getMags(colour):
    return colour[:colour.index('-')], colour[colour.index('-')+1:]
sdss = fitsParser(includefile = 'FeHdata.txt')
youc = sdss.getAllObj()[1]
cut(youc, metThresh = -5)
youc.data = youc.data[youc.data['FeHadop']<-2.0]
youc.makeColours(N=10)
colours = youc.colours
colours = ['CaHK_0-g_fpsf', 'g_fpsf-i_fpsf', 'g_fpsf-r_fpsf']
conversion = {
     'u_0' : 'u',
    'CaHK_0' : 'CaHK' 
}
conversion.update({x+'_fpsf' : x for x in 'grizy'})
for col in youc.colours:
    mag1, mag2 = getMags(col)
    youc.data[conversion[mag1]+'-'+conversion[mag2]] = youc.data[col]

for field in youc.data.keys():
    if field in conversion:
        youc.data[conversion[field]]= youc.data[field]
youc.data = youc.data.sample(frac=1)


youc.data['test_flag'] = [0]*int(0.8*len(youc.data))+[1]*(len(youc.data)-int(0.8*len(youc.data)))

classes = [-2 - binsize*i for i in range(int((3-2)/binsize)+1)]     

putbin = lambda data : [x + min([c-x for c in classes if c>x]) if x<-2 else -2 for x in data]
youc.data['FeHround'] = putbin(youc.data['FeHadop'])



def monte(din, N=3):
    tous = pd.DataFrame(din)
    uncert = pd.DataFrame()
    for col in youc.colours:
        c1 = 'd'+re.compile('.+(?=-)').findall(col)[0]
        c2 = 'd' + re.compile('(?<=-).+').findall(col)[0]
        uncert[col] = np.sqrt(tous[c1]**2+tous[c2]**2)

    uncert[map(lambda x : x[1:], youc.output_uncert)] = tous[youc.output_uncert]
    return monte_carlo(tous[list(youc.colours+youc.outputs)],uncert ,N=N)

montestyle = 2


maxlen = len(youc.data[youc.data['FeHround']==classes[0]])
minlen = min([len(youc.data[youc.data['FeHround']==c]) for c in classes])



if montestyle == 1:
    dprelim = youc.data[(youc.data['FeHround']==classes[0])&(youc.data['test_flag']==0)]
    train = dprelim
    for c in classes[1:]:
        datac = pd.DataFrame(youc.data[(youc.data['FeHround']==c)&(youc.data['test_flag']==0)])
        l = len(datac)
        print 'Num stars in class %.1f: %d'%(c,l)
        if l!=0:
            datac = monte(datac, N = maxlen/len(datac)-1)
            train = train[datac.keys()]
            train = train.append(datac, sort=True).reset_index(drop=True)
if montestyle == 2:
    nmonte = 10
    dprelim  = youc.data[youc.data['test_flag']==0]
    dprelim = monte(dprelim, N = nmonte )
    train = pd.DataFrame()
    dprelim['FeHround'] = putbin(dprelim['FeHadop'])
    for c in classes:
        toapp =  dprelim[dprelim['FeHround']==c]
        train = train.append(toapp.sample(frac = nmonte*1.0*minlen/len(toapp) if nmonte*1.0*minlen/len(toapp)<1 else 1))


train['FeHround'] = [-3 if x<=-3 else -2.5 if x<=-2.5 else -2 for x in train['FeHadop']]
train = train.sample(frac = 1).reset_index(drop= True)
for c in classes:
        datac = pd.DataFrame(train[(train['FeHround']==c)])
        l = len(datac)
        print 'Num stars in class %.1f: %d'%(c,l)

Using TensorFlow backend.



Data saved

condition on dg_fpsf cut 1 inputs
condition on dr_fpsf cut 0 inputs
condition on di_fpsf cut 0 inputs
condition on dz_fpsf cut 0 inputs
condition on dy_fpsf cut 0 inputs
condition on dCaHK_0 cut 0 inputs
condition on du_0 cut 0 inputs
condition on dFeHadop cut 17 inputs
3.13% of inputs cut
Num stars in class -2.0: 460
Num stars in class -2.5: 460
Num stars in class -3.0: 383


In [2]:
class transformer_man:
    
    def __init__(self, classes, func, inverse_func):
        self.classes = classes
        self.func = func
        self.inverse_func = inverse_func

    def transform(self,data):
        return [[1 if self.func(x, c) else 0 for c in self.classes] for x in data]
    
    def inverse_transform(self, trans_data):
        return [self.inverse_func(x) for x in trans_data]
def savemodel(model, params, data, filename):
    joblib.dump(model, filename+'.P')
    joblib.dump(params,  filename+"Params.P")
    joblib.dump(data, filename + "Data.P")

In [3]:
transformer = transformer_man([-2,-2.5,-3],
    lambda x, c :  x == c, 
    lambda x : -2-binsize*x.index(1))

youc.data['label'] = transformer.transform(youc.data['FeHround'])
train['label'] = transformer.transform(train['FeHround'])
model = KMeans()
model.fit(train[colours])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [4]:

youc.data['FeH_pred']  = model.predict(youc.data[colours])
means = {x : np.mean(youc.data[youc.data['FeH_pred']==x]['FeHadop']) for x in  set(youc.data['FeH_pred'])}
youc.data['FeH_pred'] = [means[x] for x in youc.data['FeH_pred']]
print youc.data['FeH_pred']
print means
youc.data['FeH'] = youc.data['FeHadop']
youc.data['FeHadop'] = youc.data['FeHround']
youc.data['FeHround_prist'] = putbin(youc.data['FeH_pristine'])
i = list(classes)


data = youc.data
pur, com = psMetric(data, inc = i)
print printTable(i ,pur,com)
print pd.DataFrame(conf_matrix(data, classes), columns = classes, index = classes) , '\n'      
             
pur2, com2 = psMetric(data[data['test_flag']==1], inc = i)
print printTable(i,pur2,com2)
print pd.DataFrame(conf_matrix(data[data['test_flag']==1], classes), columns = classes, index = classes),'\n'  


pur3, com3 = psMetric(data, inc = i, pred = 'FeHround_prist')
print printTable(i,pur3,com3)
print pd.DataFrame(conf_matrix(data, classes, pred = 'FeHround_prist'), columns = classes, index = classes),'\n'  



353   -2.441504
315   -2.486432
374   -2.486432
86    -2.445639
234   -2.445639
60    -2.341127
50    -2.486432
72    -2.509393
81    -2.486432
388   -2.486432
159   -2.486432
210   -2.341127
438   -2.441504
150   -2.486432
116   -2.445639
683   -2.486432
442   -2.441504
51    -2.509393
551   -2.445639
611   -2.445639
29    -2.445639
170   -2.486432
68    -2.341127
450   -2.486432
306   -2.445639
93    -2.486432
548   -2.486432
587   -2.486432
322   -2.486432
596   -2.486432
         ...   
257   -2.486432
407   -2.445639
9     -2.486432
696   -2.486432
499   -2.486432
12    -2.486432
112   -2.123733
577   -2.486432
27    -2.486432
522   -2.441504
689   -2.486432
182   -2.509393
147   -2.147200
123   -2.341127
387   -2.013550
285   -2.509393
222   -2.441504
343   -2.341127
252   -2.509393
398   -2.013550
584   -2.486432
571   -2.445639
202   -2.341127
155   -2.441504
490   -2.445639
392   -2.441504
264   -2.341127
347   -2.441504
545   -2.486432
573   -2.509393
Name: FeH_pred, Length: 

In [5]:
for c in classes:    
    print len(train[train['FeHround']==c]), c

460 -2.0
460 -2.5
383 -3.0


In [6]:
def save(model, colours, classes, binsize, data, filename):
    try:
        joblib
    except NameError:
        import joblib
    
    joblib.dump(model, filename+".P")
    joblib.dump(
    {
        'colours' : colours,
        'classes' : classes,
        'binsize' : binsize
    }, filename+'Params.P'
    )
    joblib.dump(data, filename+'Data.P')

In [7]:
#save(model, colours, classes, binsize, youc.data, 'april16v3')
print sum([ 1 if x else 0 for x in youc.data['FeH_pred']==-3])

0
